## Playground

### Todo
- eine Funktion die ein Lat/Long tuple annimmt und die Urls der n nächsten Radiostationen zurückgibt.
- eine Funktion die die Koordinaten aus dem erhaltenen Format zu Lat/Long übersetzt
- eine Funktion die die Url einer gewählten Radiostation annimmt und den Audiostream dieser Station ausgibt
- eine Funktion die den Videostream einer Webcam annimmt und Koordinaten ausliest (auf welche Art auch immer).

In [1]:
import json
import requests